#### Apply TinyML techniques to reduce model sizes

Optimize the model using TensorFlow Lite and quantization. Convert the trained model to the TensorFlow Lite format and then apply quantization to reduce its size.

In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib

import warnings
# Suppress all warnings
warnings.filterwarnings("ignore")

In [2]:
# Load back X_train, X_test, y_train, and y_test
X_train = np.loadtxt('train_data/X_train_t10.txt')
X_test = np.loadtxt('train_data/X_test_t10.txt')
y_train = np.loadtxt('train_data/y_train_t10.txt')
y_test = np.loadtxt('train_data/y_test_t10.txt')

In [3]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save the scaler for later use in real-time data normalization
joblib.dump(scaler, 'models/scaler.pkl')

['models/scaler.pkl']

Create a Deep Neural-Networks

In [4]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    # tf.keras.layers.Dropout(0.5),  # Add dropout for regularization
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',  # Change optimizer to Adam
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [5]:
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))

#### Model Training

In [6]:
model.fit(X_train_scaled, y_train, 
          epochs=100, 
          batch_size=32, 
          validation_split=0.2, 
          class_weight=class_weights,
          callbacks=[early_stopping])

Epoch 1/100
40/40 [==============================] - 1s 9ms/step - loss: 0.0419 - accuracy: 0.9844 - val_loss: 1.5194e-04 - val_accuracy: 1.0000
Epoch 2/100
40/40 [==============================] - 0s 4ms/step - loss: 5.3186e-04 - accuracy: 1.0000 - val_loss: 3.7987e-07 - val_accuracy: 1.0000
Epoch 3/100
40/40 [==============================] - 0s 4ms/step - loss: 8.1406e-06 - accuracy: 1.0000 - val_loss: 3.2672e-07 - val_accuracy: 1.0000
Epoch 4/100
40/40 [==============================] - 0s 4ms/step - loss: 3.4758e-06 - accuracy: 1.0000 - val_loss: 3.3898e-07 - val_accuracy: 1.0000
Epoch 5/100
40/40 [==============================] - 0s 7ms/step - loss: 2.5449e-06 - accuracy: 1.0000 - val_loss: 3.6199e-07 - val_accuracy: 1.0000
Epoch 6/100
40/40 [==============================] - 0s 7ms/step - loss: 1.8026e-06 - accuracy: 1.0000 - val_loss: 3.5566e-07 - val_accuracy: 1.0000
Epoch 7/100
40/40 [==============================] - 0s 6ms/step - loss: 1.4938e-06 - accuracy: 1.0000 - val_l

Model Evaluation

In [7]:
y_pred = (model.predict(X_test_scaled) > 0.5).astype(int)

13/13 [==============================] - 0s 2ms/step


In [8]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       199
         1.0       1.00      1.00      1.00       201

    accuracy                           1.00       400
   macro avg       1.00      1.00      1.00       400
weighted avg       1.00      1.00      1.00       400



In [9]:
# Convert and save the TensorFlow model to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\theor\AppData\Local\Temp\tmpfh9r69o_\assets


Save Model

In [10]:
# Save the TensorFlow Lite model to a file
model_file_path = os.path.join('models', 'model.tflite')
with open(model_file_path, "wb") as f:
    f.write(tflite_model)

In [11]:
model_size = os.path.getsize(model_file_path)
print("Model is %d bytes" % model_size)

Model is 13591972 bytes


Optimize the model

In [12]:
# Set optimizations (quantization)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
# Convert the model
tflite_model_optimize = converter.convert()

model_file_path = os.path.join('models', 'optimize.tflite')
with open(model_file_path, "wb") as f:
    f.write(tflite_model_optimize)

INFO:tensorflow:Assets written to: C:\Users\theor\AppData\Local\Temp\tmp0t_pr1pg\assets


INFO:tensorflow:Assets written to: C:\Users\theor\AppData\Local\Temp\tmp0t_pr1pg\assets


In [13]:
optimized_model_size = os.path.getsize(model_file_path)
print("Optimized Model is %d bytes" % optimized_model_size)

Optimized Model is 3401544 bytes


Evaluate the optimized model

In [14]:
# Load the optimized TensorFlow Lite model
optimized_model_path = 'models/optimize.tflite'
interpreter = tf.lite.Interpreter(model_path=optimized_model_path)
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare input data
input_data = X_test_scaled[0:1].astype(np.float32)  # Take the first sample from X_test_scaled

# Set input tensor
interpreter.set_tensor(input_details[0]['index'], input_data)

# Run inference
interpreter.invoke()

# Get the output tensor
output_data = interpreter.get_tensor(output_details[0]['index'])

# Convert output to binary predictions
predictions_binary = (output_data > 0.5).astype(int)

# Evaluate the predictions
print(classification_report(y_test[0:1], predictions_binary))

              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1

